# Prompt Engineering and Chat Completion
This is a simple tutorial that teaches prompt engineering basics and analyzes Twitter data using OpenAI large language models (LLMs).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it securely. This tutorial uses [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLMs predict the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates the quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- openai: call the OpenAI APIs.

In [3]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI
from IPython.display import display, Image, Markdown
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an interesting place because it's not located within any state; instead, it's a separate federal district. Washington, D.C. is home to important sites like the White House, the Capitol Building, and the Supreme Court. If you ever get a chance to visit, there are lots of museums and monuments to check out as well!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

it is 33


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

In [9]:
import json

with open('tweet_text.json', 'r', encoding='utf-8') as f:
    tweet_data = json.load(f)

In [10]:
print("Number of tweets:", len(tweet_data))
print("Sample tweet:", tweet_data[0])

Number of tweets: 182
Sample tweet: {'_id': {'$oid': '68e535897b92a174c5bf0554'}, 'tweet': {'text': '🤖⚠️ En IA generativa, copiar prompts puede alterar modelos y filtrar info sensible. @ZorayaMejia aconseja: crea tus propios prompts, verifica fuentes y educa al equipo. Precaución = profesionalismo.💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9'}}


### Summarization 
- Analyze tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [11]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets cover a wide range of topics related to generative AI, including its impact on various industries, ethical considerations, and practical applications. Key points include:

1. **Generative AI in Industries**: Tweets discuss the role of generative AI in transforming creative industries, finance, and local businesses, highlighting its potential to innovate and improve efficiency.

2. **Ethical and Security Concerns**: There are concerns about the ethical implications and security risks of generative AI, such as the potential for data leaks and the need for responsible AI use.

3. **AI in Content Creation**: Generative AI is being used to create content across different media, including videos, music, and marketing materials, with discussions on its impact on intellectual property and creative careers.

4. **AI Tools and Education**: Tweets mention the importance of creating custom AI prompts, educating teams on AI use, and the development of AI tools for specific applications like gene expression analysis and enterprise solutions.

5. **Environmental and Social Impact**: Some tweets express concerns about the environmental impact of AI, such as increased energy consumption, and the social implications, like potential job displacement and changes in work motivation.

6. **AI Events and Innovations**: There are mentions of AI-related events, new AI tools, and innovations, such as AI-powered learning platforms and AI agents in business.

Overall, the tweets reflect a mix of enthusiasm for the potential of generative AI and caution regarding its ethical, environmental, and social impacts.

In [13]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

Generative AI is transforming industries, raising ethical concerns, and impacting creativity, coding, and business innovation.

In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people learn AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets highlight various aspects of learning AI, emphasizing the importance of creating original prompts, understanding AI's impact on industries, and the need for ethical considerations. They also discuss AI's role in transforming workflows, the significance of upskilling, and the potential challenges and opportunities AI presents for future careers.

### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [15]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [16]:
for tweet in tweet_data:
    # print(tweet['tweet']['text'])
    flag_help(tweet['tweet']['text'])

===
harassment
Pisses me off when someone makes fun of ppl for having a lot of online friends like ALL MY IRLS ARE HOMOPHOBIC LOVE GENERATIVE AI AND PASS RACIAL SLURS AROUND LIKE A COOKIE JAR 😭🤞


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [17]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ 在生成式人工智能中，复制提示可能会改变模型并泄露敏感信息。@ZorayaMejia 建议：创建你自己的提示，核实来源并教育团队。谨慎 = 专业。💡 #人工智能 #生成式AI #数字安全 https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: 德勤澳大利亚将部分退还44万美元，因为使用AI生成的虚假报价。#生成式AI #机器学习 #伦理
RT @jinwashu: 很高兴向大家介绍我们的新工具GAiN，它使用#生成式AI进行基因表达分析，来自放射肿瘤学的@MikeWaters121…
💻 第24天/365天 – JS 函数 学习了如何： 🔹 声明和调用函数 🔹 使用参数和返回值 🔹 编写箭头函数和回调函数 ✨ 函数 = 可重用 + 更简洁的JS代码！ #100天编码 #JavaScript #MERNStack #Web开发 #生成式AI #编码军团 https://t.co/ZRbl6RQXla
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出了Chatgpt和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们应该远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频…
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频…
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频…


In [18]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 Oh, do pay attention, you simpletons! In the realm of generative AI, pilfering prompts can lead to model corruption and the leaking of sensitive information. The ever-wise @ZorayaMejia suggests crafting your own prompts, verifying sources, and educating your team. Remember, caution is the hallmark of true professionalism. 💡 #IA #GenerativeAI #DigitalSecurity 
Oh, how delightfully amusing! It seems Deloitte Australia is in a bit of a pickle, having to cough up $440,000 because of some AI-generated tomfoolery. One must wonder if their AI has been taking lessons from Brian. #GenerativeAi #MachineLearning #Ethics
Ah, splendid! Allow me to introduce you to our latest contraption, GAiN, a delightful little tool that employs #GenerativeAI for gene expression analysis. Courtesy of the ever-brilliant RadOnc and the esteemed @MikeWaters121. Do try to keep up, won't you?
💻 Day 24/365 – JS Functions
Ah, the art of mastering functions, my dear simpletons. Today, I delved into the world of:
🔹 Decla

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract the mentioned people's or organization's names
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [19]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["@ZorayaMejia"]
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Deloitte Australia", "HaywaaWorldwide"]
}
{
  "sentiment": "positive",
  "emotion": "gladness",
  "mentioned": ["jinwashu", "RadOnc", "MikeWaters121"]
}
{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": []
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}


In [20]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Generative AI and Security Concerns",
    "AI in Finance and Banking",
    "Generative AI in Creative Industries",
    "AI and Environmental Impact",
    "AI in Education and Upskilling",
    "AI Regulation and Ethics",
    "AI in Content Creation and Marketing",
    "AI in Software Development and Coding",
    "AI in Entertainment and Media",
    "AI in Business and Innovation"
  ]
}


### Expanding with multiple prompts 
- Identify topics
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [21]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative, or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 182/182 [02:43<00:00,  1.11it/s]


In [22]:
print(analysis_result)

['{"sentiment": "neutral", "emotion": "caution", "mentioned": ["@ZorayaMejia"]}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["HaywaaWorldwide", "Deloitte Australia"]\n}', '{\n  "sentiment": "positive",\n  "emotion": "gladness",\n  "mentioned": ["jinwashu", "MikeWaters121"]\n}', '{\n  "sentiment": "positive",\n  "emotion": "enthusiasm",\n  "mentioned": []\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "me

In [23]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis result delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets mentioned by people or organizations;
                                        step 2 {delimiter} identify the common sentiments and emotions for each mentioned person or organization;
                                        step 3 {delimiter} Organize the result in a json document with keys <people or organization name> 
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "@ZorayaMejia": {
    "count": 1,
    "sentiments": ["neutral"],
    "emotions": ["caution"]
  },
  "HaywaaWorldwide": {
    "count": 2,
    "sentiments": ["neutral", "negative"],
    "emotions": ["informative", "concern"]
  },
  "Deloitte Australia": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative", "none"]
  },
  "jinwashu": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "MikeWaters121": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "AmandaDannielle": {
    "count": 38,
    "sentiments": ["neutral"],
    "emotions": ["informative", "caution"]
  },
  "Chatgpt": {
    "count": 34,
    "sentiments": ["neutral"],
    "emotions": ["informative", "caution"]
  },
  "@myexamcloud": {
    "count": 7,
    "sentiments": ["neutral"],
    "emotions": ["informative", "anticipation"]
  },
  "Ronald_vanLoon": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative"

## Create a chatbot

In [24]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chatbot that answers user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people or organization name in the {delimiter}{analysis_summary}{delimiter}, report the corresponding sentiment and emotion
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [26]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    display(Markdown(f"Chatbot: {reply}"))

You:  summarize what people say about Nivida 


Chatbot: In the tweets, Nvidia is mentioned positively once, associated with excitement, particularly in relation to its impact on the stock market valuation of tech companies.

You:  Write a promt that translates the nivida tweet into ten differnt langauges


Chatbot: Here is a prompt designed to translate the Nvidia tweet into ten different languages:

---

**Prompt:**

Translate the following tweet about Nvidia into ten different languages: French, Spanish, German, Chinese, Japanese, Russian, Arabic, Portuguese, Italian, and Hindi.

**Tweet:**
"Nvidia crushes the stock market valuation of tech companies! #AI #MachineLearning #DeepLearning #DataScience #GenerativeAI"

**Desired Translations:**
1. **French:** 
2. **Spanish:**
3. **German:**
4. **Chinese:**
5. **Japanese:**
6. **Russian:**
7. **Arabic:**
8. **Portuguese:**
9. **Italian:**
10. **Hindi:**

---

Please ensure each translation maintains the original meaning and tone of excitement present in the tweet.

You:  now translate the nivida tweet into ten different languages 


Chatbot: Here are translations of the Nvidia tweet into ten different languages:

1. **French:** Nvidia écrase la valorisation boursière des entreprises technologiques ! #IA #ApprentissageAutomatique #ApprentissageProfond #ScienceDesDonnées #IAgénératrice

2. **Spanish:** ¡Nvidia aplasta la valoración bursátil de las empresas tecnológicas! #IA #AprendizajeAutomático #AprendizajeProfundo #CienciaDeDatos #IAgenerativa

3. **German:** Nvidia zerquetscht die Börsenbewertung von Technologieunternehmen! #KI #MaschinellesLernen #TiefesLernen #Datenwissenschaft #GenerativeKI

4. **Chinese (Simplified):** 英伟达重创科技公司的股票市值！#人工智能 #机器学习 #深度学习 #数据科学 #生成式人工智能

5. **Japanese:** Nvidiaがテクノロジー企業の株式市場評価を圧倒！#AI #機械学習 #深層学習 #データサイエンス #生成AI

6. **Russian:** Nvidia сокрушает рыночную оценку технологических компаний! #ИИ #МашинноеОбучение #ГлубокоеОбучение #НаукаОДанных #ГенеративныйИИ

7. **Arabic:** نيفيديا تسحق تقييم سوق الأسهم لشركات التكنولوجيا! #الذكاء_الاصطناعي #تعلم_الآلة #التعلم_العميق #علوم_البيانات #الذكاء_الاصطناعي_التوليدي

8. **Portuguese:** Nvidia esmaga a valorização do mercado de ações das empresas de tecnologia! #IA #AprendizagemAutomática #AprendizagemProfunda #CiênciaDeDados #IAgenerativa

9. **Italian:** Nvidia schiaccia la valutazione di mercato delle aziende tecnologiche! #IA #ApprendimentoAutomatico #ApprendimentoProfondo #ScienzaDeiDati #IAgenerativa

10. **Hindi:** एनवीडिया ने टेक कंपनियों के स्टॉक मार्केट मूल्यांकन को पछाड़ दिया! #एआई #मशीनलर्निंग #डीपलर्निंग #डेटासाइंस #जनरेटिवएआई

These translations aim to retain the enthusiasm and context of the original Nvidia tweet.

You:  exit


Chatbot: Goodbye!


## 🧠 Try It Yourself: Prompt Engineering Practice
👉 Be sure to **run your own cells** and show the outputs before uploading the notebook to GitHub.

1. **Summarize your tweets differently**  
   Change your system message to summarize tweets from a new perspective —  
   for example:  
   *“Summarize what people say about AI in education.”*  
   or  
   *“Summarize how people feel about the election.”*

In [27]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people feel about Nivida, 
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

People are expressing mixed feelings about Nvidia. Some tweets highlight Nvidia's significant impact on the stock market and its role in advancing AI technologies, while others raise concerns about the environmental impact and ethical implications of generative AI, which Nvidia is heavily involved in.

2. **Translate tweets**  
   Write a prompt that translates 10 tweets into another language,  
   such as Chinese, Spanish, or French.


In [28]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Spanish"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))


🤖⚠️ In generative AI, copying prompts can alter models and leak sensitive information. @ZorayaMejia advises: create your own prompts, verify sources, and educate the team. Caution = professionalism.💡 #AI #GenerativeAI #DigitalSecurity https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: Deloitte Australia reembolsará parcialmente $440,000 debido a citas falsas generadas por IA. #GenerativeAi #MachineLearning #Eth…
RT @jinwashu: Me complace presentarles nuestra nueva herramienta GAiN que utiliza #GenerativeAI para el análisis de expresión génica, de RadOnc @MikeWaters121…
💻 Día 24/365 – Funciones en JS  
Aprendí a:  
🔹 Declarar y llamar funciones  
🔹 Usar parámetros y retornos  
🔹 Escribir funciones flecha y de callback  

✨ ¡Funciones = código JS reutilizable y más limpio!

#100DaysOfCode #JavaScript #MERNStack #WebDev #GenerativeAI #coderarmy https://t.co/ZRbl6RQXla
RT @AmandaDannielle: La gente sigue diciéndoles de qué tipos de IA deben mantenerse alejados. Han sido MUY claros en que Chatgpt

3. **Simplify or rewrite tweets**  
   Ask the model to rewrite 10 tweets in a more polite, formal, or funny tone.

In [29]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""Rewrite the tweets delimited by {delimiter} to make them funnier in English. Keep the original meaning but add humor, wit, or playful exaggeration. Preserve emojis and formatting if present."""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))


🤖⚠️ En el mundo de la IA generativa, copiar prompts es como jugar al teléfono descompuesto: puedes terminar con un modelo que cree que los gatos son los verdaderos gobernantes del mundo. 🐱👑 @ZorayaMejia sugiere: inventa tus propios prompts, verifica que tus fuentes no sean de la Edad de Piedra y educa a tu equipo para que no confunda un algoritmo con una receta de cocina. Precaución = profesionalismo, y menos caos felino.💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: Deloitte Australia is set to refund $440,000 because their AI decided to moonlight as a creative writer. Who knew robots could be so... *quote-tastic*? 🤖💸 #GenerativeAi #MachineLearning #Eth...
RT @jinwashu: Excited to introduce GAiN, our new tool that uses #GenerativeAI for gene expression analysis! It's so advanced, it can probably predict what you'll have for breakfast tomorrow. From RadOnc @MikeWaters121…
💻 Day 24/365 – JS Functions
Today I discovered the magical world of:
🔹 Declarin

4. **Find keywords**  
   Design a prompt that extracts the top five most common keywords  
   from your tweet text and returns them as a list.

In [38]:
for tweet in tweet_data:
    all_text += tweet['tweet']['text'] + "\n"

messages = [
    {"role": "system", "content": "Identify the top five most common and meaningful keywords in the tweet text."},
    {"role": "user", "content": all_text}
]

response = openai_help(messages)
keywords = response.strip().split("\n")[:5]

print(keywords)


['1. GenerativeAI', '2. AI', '3. MachineLearning', '4. Ethics', '5. Innovation']


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
